In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import Data.List
import Data.Function (on)
import qualified Data.Attoparsec.ByteString.Char8 as A
import qualified Data.Vector.Unboxed as V

input <- dayLines 20

In [2]:
parseTriple = do
    A.char '<'
    x <- A.signed A.decimal
    A.char ','
    y <- A.signed A.decimal
    A.char ','
    z <- A.signed A.decimal
    A.char '>'
    return (x,y,z)

parseLine = do
    p <- A.string "p=" *> parseTriple
    A.string ", "
    v <- A.string "v=" *> parseTriple
    A.string ", "
    a <- A.string "a=" *> parseTriple
    return (p,v,a)

points = V.fromList $ map (parsed parseLine) input

In [3]:
add (x,y,z) (x',y',z') = (x+x',y+y',z+z')

step (p,v,a) = let
    v' = v `add` a
    p' = p `add` v'
    in (p',v',a)

distance ((x,y,z),_,_) = (abs x) + (abs y) + (abs z)

In [4]:
type Triple = (Int, Int, Int)
type Point = (Triple, Triple, Triple)

loop :: V.Vector Point -> Int -> V.Vector Point
loop ps 0 = ps
loop ps n = let
    ps' = V.map step ps
    in loop ps' (n-1)

In [5]:
V.minIndexBy (compare `on` distance) $ loop points 1000

308

In [6]:
position (p,_,_) = p

loopCollisions :: [Point] -> Int -> [Point]
loopCollisions ps 0 = ps
loopCollisions ps n = let
    groupedPs  = groupBy ((==) `on` position) $ sortOn position ps
    filteredPs = concat $ filter (\l -> length l == 1) groupedPs
    in loopCollisions (map step filteredPs) (n-1)

length $ loopCollisions (V.toList points) 100

504